In [18]:
import sys
sys.path.append('../')

from grow.reservoir import Reservoir
from measure.tasks import narmax
import numpy as np
from measure.metrics import get_metrics
import networkx as nx
import pandas as pd
from tqdm import tqdm

NRMSE = lambda y,y_fit: np.mean(((y-y_fit)**2)/np.var(y))

sepctral_radius = lambda W: np.max(np.abs(np.linalg.eigvals(W)))

def get_stats(reservoir: Reservoir, order):

    u, y = narmax(order=10)
    reservoir.reset()
    reservoir.train(u, y)

    predictions = reservoir.bipolar().train(u, y)
    err = np.nan if predictions is None else np.min((NRMSE(y[:, reservoir.washout:], predictions), 1))

    kr, gm = get_metrics(reservoir)

    kr /= reservoir.size()

    sr = sepctral_radius(reservoir.A)

    G = nx.DiGraph(reservoir.A)

    degrees = [d for _, d in G.degree()]

    return {
        'kr': kr,
        'gm': gm,
        'err': err,
        'sr': sr,
        'degree': np.mean(degrees),
        'size': reservoir.size(),
        'density': nx.density(G)
    }


In [ ]:
def generate_random_p(median, iqr, size=1):
    
    q1 = median - (iqr / 2)
    q3 = median + (iqr / 2)
    
    std_dev = iqr / 1.35 
    
    p_values = np.random.normal(loc=median, scale=std_dev, size=size)
    
    p_values = np.clip(p_values, 0, 1)
    
    return p_values

In [42]:
n = 100
median = 0.015
iqr = 0.010

results = []

with tqdm(total=150, desc="Generating Reservoirs", dynamic_ncols=True) as pbar:
    for i in range(150):
        p = generate_random_p(median, iqr)[0]
        A = np.random.choice([0, 1], size=(n, n), p=[1 - p, p])  
        S = np.zeros((n, 3), dtype=int)
        S[:, 0] = 1  
        reservoir = Reservoir(A, S)
        stats = get_stats(reservoir, i)
        stats["trial"] = i
        results.append(stats)

        pbar.set_postfix({"Current Error": f"{stats['err']:.3f}"})
        pbar.update(1)

stats_df = pd.DataFrame(results)

summary_stats = stats_df.agg(
    median_KR=("kr", "median"),
    IQR_KR=("kr", lambda x: x.quantile(0.75) - x.quantile(0.25)),

    median_GM=("gm", "median"),
    IQR_GM=("gm", lambda x: x.quantile(0.75) - x.quantile(0.25)),

    median_SR=("sr", "median"),
    IQR_SR=("sr", lambda x: x.quantile(0.75) - x.quantile(0.25)),

    median_Degree=("degree", "median"),
    IQR_Degree=("degree", lambda x: x.quantile(0.75) - x.quantile(0.25)),

    median_Size=("size", "median"),
    IQR_Size=("size", lambda x: x.quantile(0.75) - x.quantile(0.25)),

    median_Density=("density", "median"),
    IQR_Density=("density", lambda x: x.quantile(0.75) - x.quantile(0.25)),

    median_Error=("err", "median"),
    IQR_Error=("err", lambda x: x.quantile(0.75) - x.quantile(0.25))
).reset_index()

summary_stats

Generating Reservoirs: 100%|██████████| 150/150 [34:42<00:00, 13.88s/it, Current Error=0.502]


,index,kr,gm,sr,degree,size,density,err
0,median_KR,0.7200,NaN,NaN,NaN,NaN,NaN,NaN
1,IQR_KR,0.3775,NaN,NaN,NaN,NaN,NaN,NaN
2,median_GM,NaN,1.000526,NaN,NaN,NaN,NaN,NaN
3,IQR_GM,NaN,0.011789,NaN,NaN,NaN,NaN,NaN
4,median_SR,NaN,NaN,1.584240,NaN,NaN,NaN,NaN
5,IQR_SR,NaN,NaN,1.120766,NaN,NaN,NaN,NaN
6,median_Degree,NaN,NaN,NaN,3.190,NaN,NaN,NaN
7,IQR_Degree,NaN,NaN,NaN,2.105,NaN,NaN,NaN
8,median_Size,NaN,NaN,NaN,NaN,100.0,NaN,NaN
9,IQR_Size,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [52]:
n = 200
median = 0.010
iqr = 0.009

results = []

with tqdm(total=150, desc="Generating Reservoirs", dynamic_ncols=True) as pbar:
    for i in range(150):
        p = generate_random_p(median, iqr)[0]
        A = np.random.choice([0, 1], size=(n, n), p=[1 - p, p])  
        S = np.zeros((n, 3), dtype=int)
        S[:, 0] = 1  
        reservoir = Reservoir(A, S)
        stats = get_stats(reservoir, i)
        stats["trial"] = i
        results.append(stats)

        pbar.set_postfix({"Current Error": f"{stats['err']:.3f}"})
        pbar.update(1)

stats_df = pd.DataFrame(results)

summary_stats = stats_df.agg(
    median_KR=("kr", "median"),
    IQR_KR=("kr", lambda x: x.quantile(0.75) - x.quantile(0.25)),

    median_GM=("gm", "median"),
    IQR_GM=("gm", lambda x: x.quantile(0.75) - x.quantile(0.25)),

    median_SR=("sr", "median"),
    IQR_SR=("sr", lambda x: x.quantile(0.75) - x.quantile(0.25)),

    median_Degree=("degree", "median"),
    IQR_Degree=("degree", lambda x: x.quantile(0.75) - x.quantile(0.25)),

    median_Size=("size", "median"),
    IQR_Size=("size", lambda x: x.quantile(0.75) - x.quantile(0.25)),

    median_Density=("density", "median"),
    IQR_Density=("density", lambda x: x.quantile(0.75) - x.quantile(0.25)),

    median_Error=("err", "median"),
    IQR_Error=("err", lambda x: x.quantile(0.75) - x.quantile(0.25))
).reset_index()

summary_stats

Generating Reservoirs: 100%|██████████| 150/150 [42:41<00:00, 17.08s/it, Current Error=1.000]


,index,kr,gm,sr,degree,size,density,err
0,median_KR,0.66500,NaN,NaN,NaN,NaN,NaN,NaN
1,IQR_KR,0.46375,NaN,NaN,NaN,NaN,NaN,NaN
2,median_GM,NaN,0.999779,NaN,NaN,NaN,NaN,NaN
3,IQR_GM,NaN,0.013642,NaN,NaN,NaN,NaN,NaN
4,median_SR,NaN,NaN,1.81322,NaN,NaN,NaN,NaN
5,IQR_SR,NaN,NaN,1.66399,NaN,NaN,NaN,NaN
6,median_Degree,NaN,NaN,NaN,3.625,NaN,NaN,NaN
7,IQR_Degree,NaN,NaN,NaN,3.570,NaN,NaN,NaN
8,median_Size,NaN,NaN,NaN,NaN,200.0,NaN,NaN
9,IQR_Size,NaN,NaN,NaN,NaN,0.0,NaN,NaN
